<a href="https://colab.research.google.com/github/jinsunghub/HPC-System-Optimization/blob/main/GEMM_Convolution_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기본 정보

In [ ]:
%%writefile basic_info.cu
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    int deviceId;
    cudaError_t err = cudaGetDevice(&deviceId);
    if (err != cudaSuccess) {
        printf("Failed to get CUDA device: %s\n", cudaGetErrorString(err));
        return -1;
    }

    cudaDeviceProp props;
    err = cudaGetDeviceProperties(&props, deviceId);
    if (err != cudaSuccess) {
        printf("Failed to get device properties: %s\n", cudaGetErrorString(err));
        return -1;
    }

    printf("--- GPU Device: %s ---\n", props.name);
    printf("Compute Capability:   %d.%d\n", props.major, props.minor);

    printf("\n--- On-Chip Memory (SM Level) ---\n");
    printf("Max Shared Memory per SM (Total L1/SMEM Pool): %zu bytes (%.0f KB)\n",
           props.sharedMemPerMultiprocessor,
           (double)props.sharedMemPerMultiprocessor / 1024.0);

    printf("Max Shared Memory per Block (Programmable Limit): %zu bytes (%.0f KB)\n",
           props.sharedMemPerBlock,
           (double)props.sharedMemPerBlock / 1024.0);

    printf("\n--- Off-Chip / Global Memory ---\n");

    printf("Total Global Memory: %zu bytes (%.2f GB)\n",
           props.totalGlobalMem,
           (double)props.totalGlobalMem / (1024.0 * 1024.0 * 1024.0));

    printf("L2 Cache Size: %d bytes (%.0f KB / %.1f MB)\n",
           props.l2CacheSize,
           (double)props.l2CacheSize / 1024.0,
           (double)props.l2CacheSize / (1024.0 * 1024.0));

    return 0;
}

Overwriting basic_info.cu


In [ ]:
!nvcc basic_info.cu -o basic_info -gencode arch=compute_75,code=sm_75

In [ ]:
!./basic_info

--- GPU Device: Tesla T4 ---
Compute Capability:   7.5

--- On-Chip Memory (SM Level) ---
Max Shared Memory per SM (Total L1/SMEM Pool): 65536 bytes (64 KB)
Max Shared Memory per Block (Programmable Limit): 49152 bytes (48 KB)

--- Off-Chip / Global Memory ---
Total Global Memory: 15828320256 bytes (14.74 GB)
L2 Cache Size: 4194304 bytes (4096 KB / 4.0 MB)


실습 1. Vector Addition

*   기본 실습 (데이터 할당, 이동, warp와 thread block 크기 설정, 커널 코드 작성)
*   하나의 쓰레드가 하나의 연산을 처리함을 가정
*   2D-vector addition으로 확장

In [ ]:
%%writefile vector_1D_addition.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void initializeVectors(int* a, int* b, int size) {
    srand(time(NULL));
    for (int i = 0; i < size; i++) {
        a[i] = rand() % 100;
        b[i] = rand() % 100;
    }
}

__global__ void vectorAdd(const int* a, const int* b, int* c, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
      c[idx] = a[idx] + b[idx];
    }
}

bool verifyResult(int* vector_a, int* vector_b, int* result, int size) {
    for (int i = 0; i < size; i++) {
        if (result[i] != vector_a[i] + vector_b[i]) {
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {

    int N = (argc > 1) ? atoi(argv[1]) : 1<<20;

    int* vector_a, *vector_b, *result_vector;

    vector_a = (int*)malloc(sizeof(int) * N);
    vector_b = (int*)malloc(sizeof(int) * N);
    result_vector = (int*)malloc(sizeof(int) * N);

    int* d_a, * d_b, * d_c;
    cudaMalloc(&d_a, sizeof(int) * N);
    cudaMalloc(&d_b, sizeof(int) * N);
    cudaMalloc(&d_c, sizeof(int) * N);

    initializeVectors(vector_a, vector_b, N);
    printf("Vectors initialized with %d elements each.\n", N);
    cudaMemcpy(d_a, vector_a, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, vector_b, sizeof(int) * N, cudaMemcpyHostToDevice);



    int blockSize = (argc > 2) ? atoi(argv[2]) : 256;
    printf("Using block size of %d.\n", blockSize);

    int numBlocks = (N + blockSize - 1) / blockSize;
    printf("Launching kernel with %d blocks of size %d.\n", numBlocks, blockSize);

    vectorAdd<<<numBlocks, blockSize>>>(d_a, d_b, d_c, N);

    cudaMemcpy(result_vector, d_c, sizeof(int) * N, cudaMemcpyDeviceToHost);


    if (verifyResult(vector_a, vector_b, result_vector, N)) {
        printf("Results are correct!\n");
    } else {
        printf("Results are incorrect!\n");
    }

    free(vector_a);
    free(vector_b);
    free(result_vector);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

Overwriting vector_1D_addition.cu


In [ ]:
!nvcc vector_1D_addition.cu -o vector_1D_addition -gencode arch=compute_75,code=sm_75

In [ ]:
!./vector_1D_addition 65536 16

Vectors initialized with 65536 elements each.
Using block size of 16.
Launching kernel with 4096 blocks of size 16.
Results are correct!


In [ ]:
%%writefile vector_2D_addition.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__global__ void matrixAdd(const int* a, const int* b, int* c, int width, int height) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;
    if (col < width && row < height) {
        int idx = row * width + col;
        c[idx] = a[idx] + b[idx];
    }

}

bool verifyResult(int* matrix_a, int* matrix_b, int* result, int width, int height) {
    for (int i = 0; i < width * height; i++) {
        if (result[i] != matrix_a[i] + matrix_b[i]) {
            return false;
        }
    }
    return true;
}

void initializeMatrix(int* a, int* b, int width, int height) {
    srand(time(NULL));
    for (int i = 0; i < width * height; i++) {
        a[i] = rand() % 100;
        b[i] = rand() % 100;
    }
}

int main(int argc, char** argv) {
    int width = (argc > 1) ? atoi(argv[1]) : 1024;
    int height = (argc > 2) ? atoi(argv[2]) : 1024;
    int total_elements = width * height;

    int* matrix_a, *matrix_b, *result_matrix;

    matrix_a = (int*)malloc(sizeof(int) * total_elements);
    matrix_b = (int*)malloc(sizeof(int) * total_elements);
    result_matrix = (int*)malloc(sizeof(int) * total_elements);

    int* d_a, * d_b, * d_c;

    cudaMalloc(&d_a, sizeof(int) * total_elements);
    cudaMalloc(&d_b, sizeof(int) * total_elements);
    cudaMalloc(&d_c, sizeof(int) * total_elements);

    initializeMatrix(matrix_a, matrix_b, width, height);
    printf("Matrix initialized with size %d x %d (%d elements).\n", width, height, total_elements);

    cudaMemcpy(d_a, matrix_a, sizeof(int) * total_elements, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, matrix_b, sizeof(int) * total_elements, cudaMemcpyHostToDevice);

    int blockSide = (argc > 3) ? atoi(argv[3]) : 16;

    dim3 blockSize(blockSide, blockSide);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x,
                  (height + blockSize.y - 1) / blockSize.y);

    printf("Launching kernel with Grid(%d, %d) and Block(%d, %d).\n",
           gridSize.x, gridSize.y, blockSize.x, blockSize.y);

    matrixAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, width, height);

    cudaMemcpy(result_matrix, d_c, sizeof(int) * total_elements, cudaMemcpyDeviceToHost);

    if (verifyResult(matrix_a, matrix_b, result_matrix, width, height)) {
        printf("Results are correct!\n");
    } else {
        printf("Results are incorrect!\n");
    }

    free(matrix_a);
    free(matrix_b);
    free(result_matrix);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

Overwriting vector_2D_addition.cu


In [ ]:
!nvcc vector_2D_addition.cu -o vector_2D_addition -gencode arch=compute_75,code=sm_75

In [ ]:
!./vector_2D_addition 2048 2048 16

Matrix initialized with size 2048 x 2048 (4194304 elements).
Launching kernel with Grid(128, 128) and Block(16, 16).
Results are correct!


실습 2. General Matrix-Matrix Multiplication (GEMM)

*   기본 행렬 곱 연산
*   coalesced 메모리 접근 방식
*   공유 메모리(shared memory)를 활용한 최적화
*   행렬 크기 각 요소가 적당히 크다는 것을 가정 (General)

*주의: 수행시간 측정 시, 처음 실행은 다소 느릴 수도 있기 때문에 반복 수행하여 수행 시간을 측정할 것*

In [ ]:
%%writefile naive_gemm.cu
#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void gemm_naive_kernel(const int* A, const int* B, int* C, int M, int N, int K) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;

    if (row >= M || col >= N) return;
    int C_val = 0;

    for (int k = 0; k < K; ++k) {
        C_val += A[row * K + k] * B[k * N + col];
    }

    C[row * N + col] = C_val;
}

void gemm_cpu(const int* A, const int* B, int* C, int M, int N, int K) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            int C_val = 0;
            for (int k = 0; k < K; ++k) {
                C_val += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = C_val;
        }
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = rand() % 100;
    }
}

bool verifyResult(const int* C_gpu, const int* C_cpu, int M, int N) {
    for (int i = 0; i < M * N; ++i) {
        if (C_gpu[i] != C_cpu[i]) {
            printf("Error at index %d: GPU=%d, CPU=%d\n", i, C_gpu[i], C_cpu[i]);
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {
    if (argc != 5) {
        fprintf(stderr, "usage: %s <M> <N> <K> <num_thread>\n", argv[0]);
        fprintf(stderr, "  <num_thread>: Block size (e.g., 16 for 16x16 block)\n");
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);
    int num_thread = atoi(argv[4]);

    if (M <= 0 || N <= 0 || K <= 0 || num_thread <= 0) {
        fprintf(stderr, "M, N, K, num_thread must be greater than 0.\n");
        return 1;
    }

    printf("Executing GEMM C(M,N) = A(M,K) * B(K,N)\n");
    printf("M=%d, N=%d, K=%d\n", M, N, K);
    printf("Block dimensions: %d x %d (Total %d threads/block)\n", num_thread, num_thread, num_thread * num_thread);

    size_t A_size = (size_t)M * K * sizeof(int);
    size_t B_size = (size_t)K * N * sizeof(int);
    size_t C_size = (size_t)M * N * sizeof(int);

    int* h_A = (int*)malloc(A_size);
    int* h_B = (int*)malloc(B_size);
    int* h_C_gpu = (int*)malloc(C_size);
    int* h_C_cpu = (int*)malloc(C_size);

    srand(123);
    initializeMatrix(h_A, M * K);
    initializeMatrix(h_B, K * N);

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, A_size);
    cudaMalloc(&d_B, B_size);
    cudaMalloc(&d_C, C_size);

    cudaMemcpy(d_A, h_A, A_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, B_size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 blockDim(num_thread, num_thread);
    dim3 gridDim((M + blockDim.x - 1) / blockDim.x,
                   (N + blockDim.y - 1) / blockDim.y);

    printf("Launching Kernel: gridDim(%d, %d), blockDim(%d, %d)\n", gridDim.x, gridDim.y, blockDim.x, blockDim.y);

    cudaEventRecord(start);

    gemm_naive_kernel<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, N, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("\n--- Naive Kernel Execution Time --- \n");
    printf("Time: %.4f ms\n", milliseconds);

    cudaMemcpy(h_C_gpu, d_C, C_size, cudaMemcpyDeviceToHost);

    printf("\nVerifying results...\n");
    //gemm_cpu(h_A, h_B, h_C_cpu, M, N, K);

    if (verifyResult(h_C_gpu, h_C_cpu, M, N)) {
       printf("Success: Results are correct!\n");
    } else {
        printf("Failure: Results are incorrect!\n");
    }

    free(h_A);
    free(h_B);
    free(h_C_gpu);
    free(h_C_cpu);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting naive_gemm.cu


In [ ]:
!nvcc naive_gemm.cu -o naive_gemm -gencode arch=compute_75,code=sm_75

In [ ]:
!./naive_gemm 2048 2048 1024 16

Executing GEMM C(M,N) = A(M,K) * B(K,N)
M=2048, N=2048, K=1024
Block dimensions: 16 x 16 (Total 256 threads/block)
Launching Kernel: gridDim(128, 128), blockDim(16, 16)

--- Naive Kernel Execution Time --- 
Time: 169.6749 ms

Verifying results...
Error at index 0: GPU=2597831, CPU=0
Failure: Results are incorrect!


In [ ]:
!./naive_gemm 1024 1024 1024 32

Executing GEMM C(M,N) = A(M,K) * B(K,N)
M=1024, N=1024, K=1024
Block dimensions: 32 x 32 (Total 1024 threads/block)
Launching Kernel: gridDim(32, 32), blockDim(32, 32)

--- Naive Kernel Execution Time --- 
Time: 58.7607 ms

Verifying results...
Error at index 0: GPU=2667833, CPU=0
Failure: Results are incorrect!


In [ ]:
%%writefile coalesced_gemm.cu

#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void gemm_coalesced_kernel(const int* A, const int* B, int* C, int M, int N, int K) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row >= M || col >= N) return;
    int C_val = 0;

    for (int k = 0; k < K; ++k) {
        C_val += A[row * K + k] * B[k * N + col];
    }

    C[row * N + col] = C_val;
}

void gemm_cpu(const int* A, const int* B, int* C, int M, int N, int K) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            int C_val = 0;
            for (int k = 0; k < K; ++k) {
                C_val += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = C_val;
        }
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = rand() % 10;
    }
}

bool verifyResult(const int* C_gpu, const int* C_cpu, int M, int N) {
    for (int i = 0; i < M * N; ++i) {
        if (C_gpu[i] != C_cpu[i]) {
            printf("Error at index %d: GPU=%d, CPU=%d\n", i, C_gpu[i], C_cpu[i]);
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {
    if (argc != 5) {
        fprintf(stderr, "사용법: %s <M> <N> <K> <num_thread>\n", argv[0]);
        fprintf(stderr, "  <num_thread>: 블록의 한 변 크기 (예: 16이면 16x16 블록)\n");
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);
    int num_thread = atoi(argv[4]);

    if (M <= 0 || N <= 0 || K <= 0 || num_thread <= 0) {
        fprintf(stderr, "M, N, K, num_thread는 0보다 커야 합니다.\n");
        return 1;
    }

    printf("Executing GEMM C(M,N) = A(M,K) * B(K,N)\n");
    printf("M=%d, N=%d, K=%d\n", M, N, K);
    printf("Block dimensions: %d x %d (Total %d threads/block)\n", num_thread, num_thread, num_thread * num_thread);

    size_t A_size = (size_t)M * K * sizeof(int);
    size_t B_size = (size_t)K * N * sizeof(int);
    size_t C_size = (size_t)M * N * sizeof(int);

    int* h_A = (int*)malloc(A_size);
    int* h_B = (int*)malloc(B_size);
    int* h_C_gpu = (int*)malloc(C_size);
    int* h_C_cpu = (int*)malloc(C_size);

    srand(123);
    initializeMatrix(h_A, M * K);
    initializeMatrix(h_B, K * N);

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, A_size);
    cudaMalloc(&d_B, B_size);
    cudaMalloc(&d_C, C_size);

    cudaMemcpy(d_A, h_A, A_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, B_size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 blockDim(num_thread, num_thread);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x,
                   (M + blockDim.y - 1) / blockDim.y);

    printf("Launching Kernel: gridDim(%d, %d), blockDim(%d, %d)\n", gridDim.x, gridDim.y, blockDim.x, blockDim.y);

    cudaEventRecord(start);

    gemm_coalesced_kernel<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, N, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("\n--- Coalesced Kernel Execution Time --- \n");
    printf("Time: %.4f ms\n", milliseconds);

    cudaMemcpy(h_C_gpu, d_C, C_size, cudaMemcpyDeviceToHost);

    printf("\nVerifying results...\n");
    //gemm_cpu(h_A, h_B, h_C_cpu, M, N, K);

    if (verifyResult(h_C_gpu, h_C_cpu, M, N)) {
        printf("Success: Results are correct!\n");
    } else {
        printf("Failure: Results are incorrect!\n");
    }

    free(h_A);
    free(h_B);
    free(h_C_gpu);
    free(h_C_cpu);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting coalesced_gemm.cu


In [ ]:
!nvcc coalesced_gemm.cu -o coalesced_gemm -gencode arch=compute_75,code=sm_75

In [ ]:
!./coalesced_gemm 2048 2048 1024 16

Executing GEMM C(M,N) = A(M,K) * B(K,N)
M=2048, N=2048, K=1024
Block dimensions: 16 x 16 (Total 256 threads/block)
Launching Kernel: gridDim(128, 128), blockDim(16, 16)

--- Coalesced Kernel Execution Time --- 
Time: 22.8213 ms

Verifying results...
Error at index 0: GPU=21271, CPU=0
Failure: Results are incorrect!


In [ ]:
!./coalesced_gemm 1024 1024 1024 32

Executing GEMM C(M,N) = A(M,K) * B(K,N)
M=1024, N=1024, K=1024
Block dimensions: 32 x 32 (Total 1024 threads/block)
Launching Kernel: gridDim(32, 32), blockDim(32, 32)

--- Coalesced Kernel Execution Time --- 
Time: 6.8060 ms

Verifying results...
Error at index 0: GPU=21853, CPU=0
Failure: Results are incorrect!


In [ ]:
%%writefile shared_memory_gemm.cu

#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void gemm_smem_dynamic_kernel(const int* A, const int* B, int* C, int M, int N, int K) {

    extern __shared__ int s_data[];//max 48KB

    const int TILE_DIM = blockDim.x;

    int* s_A = s_data;
    int* s_B = (int*)&s_A[TILE_DIM * TILE_DIM];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int col = blockIdx.x * TILE_DIM + tx;
    int row = blockIdx.y * TILE_DIM + ty;
    int C_val = 0;

    for (int p = 0; p < (K + TILE_DIM - 1) / TILE_DIM; ++p) {
        int a_load_col = p * TILE_DIM + tx;
        int a_load_row = row;
        if (a_load_row < M && a_load_col < K) {
            s_A[ty * TILE_DIM + tx] = A[a_load_row * K + a_load_col];
        } else {
            s_A[ty * TILE_DIM + tx] = 0;
        }
    int b_load_col = col;
    int b_load_row = p * TILE_DIM + ty;
    if (b_load_row < K && b_load_col < N) {
        s_B[ty * TILE_DIM + tx] = B[b_load_row * N + b_load_col];
    } else {
        s_B[ty * TILE_DIM + tx] = 0;
    }

    __syncthreads();
    for (int k_tile = 0; k_tile < TILE_DIM; ++k_tile) {
        C_val += s_A[ty * TILE_DIM + k_tile] * s_B[k_tile * TILE_DIM + tx];
    }
    __syncthreads();
}
if (row < M && col < N) {
    C[row * N + col] = C_val;
}
}

void gemm_cpu(const int* A, const int* B, int* C, int M, int N, int K) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            int C_val = 0;
            for (int k = 0; k < K; ++k) {
                C_val += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = C_val;
        }
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = rand() % 10;
    }
}

bool verifyResult(const int* C_gpu, const int* C_cpu, int M, int N) {
    for (int i = 0; i < M * N; ++i) {
        if (C_gpu[i] != C_cpu[i]) {
            printf("Error at index %d: GPU=%d, CPU=%d\n", i, C_gpu[i], C_cpu[i]);
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {
    if (argc != 5) {
        fprintf(stderr, "사용법: %s <M> <N> <K> <num_thread>\n", argv[0]);
        fprintf(stderr, "  <num_thread>: 타일 및 블록의 한 변 크기 (예: 16 또는 32)\n");
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);
    int num_thread = atoi(argv[4]); // TILE_DIM

    if (M <= 0 || N <= 0 || K <= 0 || num_thread <= 0) {
        fprintf(stderr, "M, N, K, num_thread는 0보다 커야 합니다.\n");
        return 1;
    }

    printf("Executing Dynamic Tiled GEMM C(M,N) = A(M,K) * B(K,N)\n");
    printf("M=%d, N=%d, K=%d\n", M, N, K);
    printf("TILE_DIM: %d x %d (Total %d threads/block)\n", num_thread, num_thread, num_thread * num_thread);

    size_t A_size = (size_t)M * K * sizeof(int);
    size_t B_size = (size_t)K * N * sizeof(int);
    size_t C_size = (size_t)M * N * sizeof(int);

    int* h_A = (int*)malloc(A_size);
    int* h_B = (int*)malloc(B_size);
    int* h_C_gpu = (int*)malloc(C_size);
    int* h_C_cpu = (int*)malloc(C_size);

    srand(123);
    initializeMatrix(h_A, M * K);
    initializeMatrix(h_B, K * N);

    int *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, A_size);
    cudaMalloc(&d_B, B_size);
    cudaMalloc(&d_C, C_size);

    cudaMemcpy(d_A, h_A, A_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, B_size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 blockDim(num_thread, num_thread);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x,
                   (M + blockDim.y - 1) / blockDim.y);

    size_t shared_mem_bytes = 2 * (size_t)num_thread * num_thread * sizeof(int);

    printf("Launching Kernel: gridDim(%d, %d), blockDim(%d, %d), sharedMem= %zu bytes\n",
           gridDim.x, gridDim.y, blockDim.x, blockDim.y, shared_mem_bytes);

    cudaEventRecord(start);

    gemm_smem_dynamic_kernel<<<gridDim, blockDim, shared_mem_bytes>>>(d_A, d_B, d_C, M, N, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("\n--- Dynamic SMEM Kernel Execution Time --- \n");
    printf("Time: %.4f ms\n", milliseconds);

    cudaMemcpy(h_C_gpu, d_C, C_size, cudaMemcpyDeviceToHost);

    printf("\nVerifying results...\n");
    //gemm_cpu(h_A, h_B, h_C_cpu, M, N, K);

    if (verifyResult(h_C_gpu, h_C_cpu, M, N)) {
       printf("Success: Results are correct!\n");
    } else {
       printf("Failure: Results are incorrect!\n");
    }

    free(h_A);
    free(h_B);
    free(h_C_gpu);
    free(h_C_cpu);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting shared_memory_gemm.cu


In [ ]:
!nvcc shared_memory_gemm.cu -o shared_memory_gemm -gencode arch=compute_75,code=sm_75

In [ ]:
!./shared_memory_gemm 1024 1024 512 16

Executing Dynamic Tiled GEMM C(M,N) = A(M,K) * B(K,N)
M=1024, N=1024, K=512
TILE_DIM: 16 x 16 (Total 256 threads/block)
Launching Kernel: gridDim(64, 64), blockDim(16, 16), sharedMem= 2048 bytes

--- Dynamic SMEM Kernel Execution Time --- 
Time: 3.8003 ms

Verifying results...
Error at index 0: GPU=10789, CPU=0
Failure: Results are incorrect!


In [ ]:
!./shared_memory_gemm 2048 2048 1024 32

Executing Dynamic Tiled GEMM C(M,N) = A(M,K) * B(K,N)
M=2048, N=2048, K=1024
TILE_DIM: 32 x 32 (Total 1024 threads/block)
Launching Kernel: gridDim(64, 64), blockDim(32, 32), sharedMem= 8192 bytes

--- Dynamic SMEM Kernel Execution Time --- 
Time: 28.5811 ms

Verifying results...
Error at index 0: GPU=21271, CPU=0
Failure: Results are incorrect!


In [ ]:
!./coalesced_gemm 4096 4096 4096 32

Executing GEMM C(M,N) = A(M,K) * B(K,N)
M=4096, N=4096, K=4096
Block dimensions: 32 x 32 (Total 1024 threads/block)
Launching Kernel: gridDim(128, 128), blockDim(32, 32)

--- Coalesced Kernel Execution Time --- 
Time: 324.3971 ms

Verifying results...
Error at index 0: GPU=83102, CPU=0
Failure: Results are incorrect!


실습 2 보고서(해당 칸에 다음의 내용을 기재하시오)

1. naive, coalesced에 대해 아래 수행시간 표(ms)와 메모리 접근 패턴 비교 표를 채워넣을 것. (팁: 수행시간 측정 시, host 수행 및 검증 코드는 주석 처리)
(숫자와 '|' 표시를 활용)
*   thread block size: (16,16)
N,M = 1024 | 1024 | 2048 | 4096 | 8192
-|-|-|-|-
naive | 48.97ms | 174.1ms | 466.25ms | 1345.4ms
coalesced | 9.28ms | 37.46ms | 140.7ms | 403.12ms

*   thread block size: (32,32)
N,M = 4096 | 4096 | 8192 | 16384 | 32768
-|-|-|-|-|
naive | 750.63ms | 2391.87ms | 9051.92ms | 35807.01ms
coalesced | 107.33ms | 298.29ms| 989.71ms | 3577.96ms

*   warp 당 메모리 접근 패턴 비교 (세부 사항은 강의자료 참고)

warp 배치 방식 | 행렬 A | 행렬 B | 행렬 C | 메모리 전송 수
-|-|-|-|-
세로 배치 | Non-coalesced | Broadcast | Non-coalesced | 33 x K + 32
가로 배치 | Broadcast | Coalesced | Coalesced |K+2

2. coalesced와 smem에 대해 아래 수행 시간(ms) 표를 채워넣고 간단히 분석할 것
*   thread block size: (16,16)
N,M = 1024 | 256 | 512 | 1024 | 2048  
-|-|-|-|-
coalesced | 0.82ms | 2.93ms | 15.32ms | 109.57ms
smem | 0.61ms | 2.04ms | 10.18ms | 74.86ms

SMEM 방법이 coalesced 대비 약 1.4배 정도 빠릅니다. 크기가 작은 행렬에서는 차이가 적지만 큰 행렬에서는 성능 향상이 명확합니다.

*   thread block size: (32,32)
N,M = 4096 | 4096 | 8192 | 16384 | 32768  
-|-|-|-|-
coalesced | 317.63ms | 1800.81ms | 15214.06ms | 145342.67ms
smem | 199.94ms | 1695.41ms | 13002.36ms | 107589.5ms

SMEM 방법이 coalesced 대비 약 1.04배 더 빠릅니다.
 32x32 블록 사이즈에서는 성능 향상이 제한적입니다. SMEM 최적화 방법은 전역 메모리 접근 횟수 감소로 타일링을 통해 데이터를 재사용했습니다. 단, 32x32 블록 사이즈에서는 SMEM 할당으로 인해 L1 캐시 공간이 감소하여 선능 향상의 제한이 있었습니다.

 결론적으로, SMEM 최적화 방법은 메모리 재사용이 많은 경우 효과적이고 Block 크기와 SMEM 사용량의 균형이 중요하다고 생각합니다. 또한 Coalesced 접근만으로도 충분한 성능을 달성할 수 있다고 생각합니다.

Prefetching이라고 다음 타일 데이터를 실행 중에 미리 로하여 메모리 대기 시간을 감소하면 더 빠른 성능을 낼 수 있다고 생각합니다.

4. Convolution 연산 구현


In [ ]:
%%writefile naive_convolution.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#define BLOCK_DIM 32

__global__ void conv_naive_kernel(const int* input, const int* kernel, int* output,
                                  int M, int N, int K) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row >= M || col >= N) return;
    int k_radius = K / 2;
    int output_val = 0;
    for (int i = -k_radius; i <= k_radius; ++i) {
        for (int j = -k_radius; j <= k_radius; ++j) {
            int in_row = row + i;
            int in_col = col + j;
            if (in_row >= 0 && in_row < M && in_col >= 0 && in_col < N) {
                int k_row = i + k_radius;
                int k_col = j + k_radius;
                output_val += input[in_row * N + in_col] * kernel[k_row * K + k_col];
            }
        }
    }
    output[row * N + col] = output_val;

}

void conv_cpu(const int* input, const int* kernel, int* output, int M, int N, int K) {
    int k_radius = K / 2;

    for (int row = 0; row < M; ++row) {
        for (int col = 0; col < N; ++col) {

            int output_val = 0;
            for (int i = -k_radius; i <= k_radius; ++i) {
                for (int j = -k_radius; j <= k_radius; ++j) {

                    int in_row = row + i;
                    int in_col = col + j;

                    if (in_row >= 0 && in_row < M && in_col >= 0 && in_col < N) {
                        int k_row = i + k_radius;

                        int k_col = j + k_radius;
                        output_val += input[in_row * N + in_col] * kernel[k_row * K + k_col];
                    }
                }
            }
            output[row * N + col] = output_val;
        }
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = rand() % 10;
    }
}

bool verifyResult(const int* C_gpu, const int* C_cpu, int M, int N) {
    for (int i = 0; i < M * N; ++i) {
        if (C_gpu[i] != C_cpu[i]) {
            printf("Error at index %d: GPU=%d, CPU=%d\n", i, C_gpu[i], C_cpu[i]);
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {
    if (argc != 4) {
        fprintf(stderr, "usage: %s <M> <N> <K_dim>\n", argv[0]);
        fprintf(stderr, "  <M>: height\n");
        fprintf(stderr, "  <N>: width\n");
        fprintf(stderr, "  <K_dim>: kernel size (odd, e.g., 3, 5, 7)\n");
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);

    if (M <= 0 || N <= 0 || K <= 0) {
        fprintf(stderr, "M, N, K_dim must be bigger than 0.\n");
        return 1;
    }
    if (K % 2 == 0) {
        fprintf(stderr, "Kernel size (K_dim) must be odd.\n");
        return 1;
    }

    printf("Executing 2D Convolution\n");
    printf("Image (M,N): (%d, %d), Kernel (K,K): (%d, %d)\n", M, N, K, K);

    size_t input_size = (size_t)M * N * sizeof(int);
    size_t kernel_size = (size_t)K * K * sizeof(int);
    size_t output_size = (size_t)M * N * sizeof(int);

    int* h_input = (int*)malloc(input_size);
    int* h_kernel = (int*)malloc(kernel_size);
    int* h_output_gpu = (int*)malloc(output_size);
    int* h_output_cpu = (int*)malloc(output_size);

    srand(123);
    initializeMatrix(h_input, M * N);
    initializeMatrix(h_kernel, K * K);

    int *d_input, *d_kernel, *d_output;

    cudaMalloc(&d_input, input_size);
    cudaMalloc(&d_kernel, kernel_size);
    cudaMalloc(&d_output, output_size);


    cudaMemcpy(d_input, h_input, input_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, kernel_size, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 blockDim(BLOCK_DIM, BLOCK_DIM);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x,
                   (M + blockDim.y - 1) / blockDim.y);

    printf("Launching Kernel: gridDim(%d, %d), blockDim(%d, %d)\n", gridDim.x, gridDim.y, blockDim.x, blockDim.y);

    cudaEventRecord(start);

    conv_naive_kernel<<<gridDim, blockDim>>>(d_input, d_kernel, d_output, M, N, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("\n--- Naive Conv Kernel Execution Time --- \n");
    printf("Time: %.4f ms\n", milliseconds);

    cudaMemcpy(h_output_gpu, d_output, output_size, cudaMemcpyDeviceToHost);

    printf("\nVerifying results...\n");
    conv_cpu(h_input, h_kernel, h_output_cpu, M, N, K);

    if (verifyResult(h_output_gpu, h_output_cpu, M, N)) {
        printf("Success: Results are correct!\n");
    } else {
        printf("Failure: Results are incorrect!\n");
    }

    free(h_input);
    free(h_kernel);
    free(h_output_gpu);
    free(h_output_cpu);
    cudaFree(d_input);
    cudaFree(d_kernel);
    cudaFree(d_output);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting naive_convolution.cu


In [ ]:
!nvcc naive_convolution.cu -o naive_convolution -gencode arch=compute_75,code=sm_75

In [ ]:
!./naive_convolution 1024 1024 5

Executing 2D Convolution
Image (M,N): (1024, 1024), Kernel (K,K): (5, 5)
Launching Kernel: gridDim(32, 32), blockDim(32, 32)

--- Naive Conv Kernel Execution Time --- 
Time: 0.2147 ms

Verifying results...
Success: Results are correct!


In [ ]:
%%writefile optimal_convolution.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#define BLOCK_DIM 32
#define MAX_K_DIM 15

// Constant Memory 선언
__constant__ int c_kernel[MAX_K_DIM * MAX_K_DIM];

__global__ void conv_smem_kernel(const int* input, int* output,
                                 int M, int N, int K) {

    extern __shared__ int smem[];

    int k_radius = K / 2;
    int tile_width = BLOCK_DIM + 2 * k_radius;  // K - 1 = 2 * k_radius

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 현재 블록 입력 데이터의 시작 위치
    int row_start = by * BLOCK_DIM - k_radius;
    int col_start = bx * BLOCK_DIM - k_radius;

    // 전역 메모리에서 공유 메모리로 데이터 이동
    for (int i = ty; i < tile_width; i += BLOCK_DIM) {
        for (int j = tx; j < tile_width; j += BLOCK_DIM) {
            int in_row = row_start + i;
            int in_col = col_start + j;

            if (in_row >= 0 && in_row < M && in_col >= 0 && in_col < N) {
                smem[i * tile_width + j] = input[in_row * N + in_col];
            } else {
                smem[i * tile_width + j] = 0;
            }
        }
    }

    __syncthreads();

    int col_out = bx * BLOCK_DIM + tx;
    int row_out = by * BLOCK_DIM + ty;

    if (row_out >= M || col_out >= N) {
        return;
    }

    int output_val = 0;

    // Convolution
    for (int ki = 0; ki < K; ++ki) {
        for (int kj = 0; kj < K; ++kj) {
            int smem_row = ty + ki;
            int smem_col = tx + kj;

            output_val += smem[smem_row * tile_width + smem_col] * c_kernel[ki * K + kj];
        }
    }

    output[row_out * N + col_out] = output_val;
}

void conv_cpu(const int* input, const int* kernel, int* output, int M, int N, int K) {
    int k_radius = K / 2;

    for (int row = 0; row < M; ++row) {
        for (int col = 0; col < N; ++col) {

            int output_val = 0;
            for (int i = -k_radius; i <= k_radius; ++i) {
                for (int j = -k_radius; j <= k_radius; ++j) {

                    int in_row = row + i;
                    int in_col = col + j;

                    if (in_row >= 0 && in_row < M && in_col >= 0 && in_col < N) {
                        int k_row = i + k_radius;
                        int k_col = j + k_radius;
                        output_val += input[in_row * N + in_col] * kernel[k_row * K + k_col];
                    }
                }
            }
            output[row * N + col] = output_val;
        }
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = rand() % 10;
    }
}

bool verifyResult(const int* C_gpu, const int* C_cpu, int M, int N) {
    for (int i = 0; i < M * N; ++i) {
        if (C_gpu[i] != C_cpu[i]) {
            printf("Error at index %d: GPU=%d, CPU=%d\n", i, C_gpu[i], C_cpu[i]);
            return false;
        }
    }
    return true;
}

int main(int argc, char** argv) {
    if (argc != 4) {
        fprintf(stderr, "usage: %s <M> <N> <K_dim>\n", argv[0]);
        fprintf(stderr, "  <M>: height\n");
        fprintf(stderr, "  <N>: width\n");
        fprintf(stderr, "  <K_dim>: kernel size (odd, e.g., 3, 5, 7)\n");
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);

    if (M <= 0 || N <= 0 || K <= 0) {
        fprintf(stderr, "M, N, K_dim must be bigger than 0.\n");
        return 1;
    }
    if (K % 2 == 0) {
        fprintf(stderr, "Kernel size (K_dim) must be odd.\n");
        return 1;
    }

    if (K > MAX_K_DIM) {
        fprintf(stderr, "Error: Kernel size %d exceeds MAX_K_DIM %d\n", K, MAX_K_DIM);
        fprintf(stderr, "Please increase MAX_K_DIM in the source and recompile.\n");
        return 1;
    }


    printf("Executing 2D Convolution\n");
    printf("Image (M,N): (%d, %d), Kernel (K,K): (%d, %d)\n", M, N, K, K);
    printf("Block DIM: (%d, %d)\n", BLOCK_DIM, BLOCK_DIM);

    size_t input_size = (size_t)M * N * sizeof(int);
    size_t kernel_size = (size_t)K * K * sizeof(int);
    size_t output_size = (size_t)M * N * sizeof(int);

    int* h_input = (int*)malloc(input_size);
    int* h_kernel = (int*)malloc(kernel_size);
    int* h_output_gpu = (int*)malloc(output_size);
    int* h_output_cpu = (int*)malloc(output_size);

    srand(123);
    initializeMatrix(h_input, M * N);
    initializeMatrix(h_kernel, K * K);

    int *d_input, *d_output;

    cudaMalloc(&d_input, input_size);
    cudaMalloc(&d_output, output_size);

    cudaMemcpy(d_input, h_input, input_size, cudaMemcpyHostToDevice);

    printf("Copying kernel to __constant__ memory...\n");
    cudaMemcpyToSymbol(c_kernel, h_kernel, kernel_size, 0, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 blockDim(BLOCK_DIM, BLOCK_DIM);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x,
                 (M + blockDim.y - 1) / blockDim.y);

    printf("Launching Kernel: gridDim(%d, %d), blockDim(%d, %d)\n", gridDim.x, gridDim.y, blockDim.x, blockDim.y);

    int tile_width = BLOCK_DIM + 2 * (K / 2);
    size_t smem_size = (size_t)tile_width * tile_width * sizeof(int);

    printf("\nLaunching SMEM Kernel with %zu bytes of dynamic shared memory.\n", smem_size);

    cudaMemset(d_output, 0, output_size);

    cudaEventRecord(start);

    conv_smem_kernel<<<gridDim, blockDim, smem_size>>>(d_input, d_output, M, N, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("\n--- Tiled Conv Kernel (SMEM + Constant) --- \n");
    printf("Time: %.4f ms\n", milliseconds);

    cudaMemcpy(h_output_gpu, d_output, output_size, cudaMemcpyDeviceToHost);

    // CPU verification
    printf("\nRunning CPU verification...\n");
    conv_cpu(h_input, h_kernel, h_output_cpu, M, N, K);

    printf("\nVerifying results (SMEM)...\n");
    if (verifyResult(h_output_gpu, h_output_cpu, M, N)) {
        printf("Success: SMEM results are correct!\n");
    } else {
        printf("Failure: SMEM results are incorrect!\n");
    }


    free(h_input);
    free(h_kernel);
    free(h_output_gpu);
    free(h_output_cpu);
    cudaFree(d_input);
    cudaFree(d_output);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting optimal_convolution.cu


In [ ]:
!nvcc optimal_convolution.cu -o optimal_convolution -gencode arch=compute_75,code=sm_75

In [ ]:
!./optimal_convolution 8192 8192 5

Executing 2D Convolution
Image (M,N): (8192, 8192), Kernel (K,K): (5, 5)
Block DIM: (32, 32)
Copying kernel to __constant__ memory...
Launching Kernel: gridDim(256, 256), blockDim(32, 32)

Launching SMEM Kernel with 5184 bytes of dynamic shared memory.

--- Tiled Conv Kernel (SMEM + Constant) --- 
Time: 13.4056 ms

Running CPU verification...

Verifying results (SMEM)...
Success: SMEM results are correct!


보고서

*  Shared Memory 타일링: 32x32 크기의 thread block이 출력 데이터를 계산하기 위해 필요한 입력 데이터를 전역 메모리를 공유 메모리로 한 번에 로드합니다. 이를 통해 동일한 입력 픽셀을 중복해서 읽는 전역 메모리 접근 횟수를 줄였습니다.

*   경계 영역 포함 로딩: 컨볼루션 반경(k_radius)으로 인해 타일 외부의 데이터가 연산에 필요합니다. 이는 thread block 크기보다 큰 tile_width(BLOCK_DIM+2 x k_Radius)를 설정하여 주변 경계 영역까지 공유 메모리에 함께 로드했습니다.

*   Constant Memory 활용: 연산 중 값이 변하지 않고 모든 thread가 동시에 접근하는 커널 가중치를 constant 메모리 커널에 할당했습니다. 이는 상수 캐시를 활용하여 읽기 대역폭을 높이고 연산 속도를 향상시키는 역할을 합니다.


2. 이미지 크기 | 커널 크기 | Naive | Optimal | 성능 향상 비율
-|-|-|-|-
4096x4096 | 3x3 | 2.13ms | 2.29ms | 약 -7.2%
4096x4096 | 7x7 | 6.0ms | 4.33ms | 약28.6%
4096x4096 | 11x11 | 13.05ms | 7.52ms | 약42.3%
1024x1024 | 5x5 | 0.34ms | 0.24ms | 약28.8%
2048x2048 | 5x5 | 1.09ms | 0.84ms | 약22.7%
8192x8192 | 5x5 | 15.03ms | 13.43ms | 약10.6%

이미지 크기가 4096x4096으로 고정되었을 때, 커널이 커질수록 성능 향상 폭이 커졌습니다. 이는 커널이 커질수록 데이터 재사용 횟수가 기하급수적으로 증가하기 때문에, 전역 메모리 접근을 줄이고 공유 메모리를 활용하는 방식이 잘 사용되었습니다.

작은 커널(k=3)에서는 최적화 버전이 Naive보다 다소 느리게 나타났습니다. k=3 수준에서는 데이터 재사용 이득보다 공유 메모리에 경계 영역을 계산하여 로드하고 thread 간 동기화를 수행하는 오버헤드가 더 크기 때문으로 생각됩니다.

최종 결론: 본 실험을 통해 Shared Memory Tiling과 Constant Memory를 결합한 최적화 방식은 연산 집약적이고 데이터 재사용이 많은 큰 연산에서 매우 효과적임을 확인했습니다. 다만, 커널 크기가 매우 작은 경우에는 오버헤드를 고려하여 Naive 방식을 사용하는 것이 유리할 수 있습니다.